In [23]:
from faiss_search import FAISSSearch

search_engine = FAISSSearch()

In [26]:
import requests
import json

# Define API URL
API_URL = "http://localhost:8001/search"

# Define search query parameters
query_payload = {
    "query": "International guidelines for hypertension",
    "limit": 10,
    "filters": {
        "yearRange": {"start": 2015, "end": 2025},
        "language": "en",
        "publisher": "Wiley"
    }
}

# Send POST request to the API
response = requests.post(API_URL, json=query_payload, headers={"Content-Type": "application/json"})

# Check if the request was successful
if response.status_code == 200:
    results = response.json()['eval_results']
    for result in results:
        print(result['title'],result['score'])

    
    # Extract fields safely using `.get()` to prevent KeyErrors
    # international_guidelines = results.get("International", [])
    # european_guidelines = results.get("European", [])
    # usa_guidelines = results.get("USA", [])
    # national_guidelines = results.get("National", [])
    
    # # Print extracted guidelines
    # print("International Guidelines:", json.dumps(international_guidelines, indent=4))
    # print("European Guidelines:", json.dumps(european_guidelines, indent=4))
    # print("USA Guidelines:", json.dumps(usa_guidelines, indent=4))
    # print("National Guidelines:", json.dumps(national_guidelines, indent=4))

else:
    print("Error:", response.status_code, response.text)

International guidelines for groin hernia management 0.559
Second International Guidelines for the Diagnosis and Management of Hereditary Hemorrhagic Telangiectasia 0.474
2020 International Society of Hypertension global hypertension practice guidelines – lifestyle modification 0.441
Peripartum management of hypertension: a position paper of the ESC Council on Hypertension and the European Society of Hypertension 0.417
Pacemakers and MRI: A protocol in line with international guidelines and approved by the SFICV (French Society of Cardiovascular Imaging) 0.379
A European Renal Association (ERA) synopsis for nephrology practice of the 2023 European Society of Hypertension (ESH) Guidelines for the Management of Arterial Hypertension 0.368
Reappraisal of the European guidelines on hypertension management. The European Society of Hypertension Task Force document: a short review 0.366
The 2021 International Society for the Study of Hypertension in Pregnancy classification, diagnosis &amp; m

In [27]:
def perform_search(query, top_k=10, filter_pre_2015=False):
    """
    Perform search and return results as a pandas DataFrame
    """
    # Get search results
    results, abstracts = search_engine.search(query, top_k=top_k, filter_pre_2015=filter_pre_2015)
    ranked_results = FAISSSearch.rank_results(results,p=0.9)
    for result in ranked_results:
        print(result['title'],result['recency_score'])

query = "International guidelines for hypertension"
perform_search(query, top_k=10)

2020 International Society of Hypertension global hypertension practice guidelines – lifestyle modification 0.5
Hypertension Canada’s 2020 Evidence Review and Guidelines for the Management of Resistant Hypertension 0.5
2018 ESC/ESH Guidelines for the management of arterial hypertension 0.4
ISFM Consensus Guidelines on the Diagnosis and Management of Hypertension in Cats 0.3
Guideline for the diagnosis and management of hypertension in adults — 2016 0.2
Guidelines for the Management of Arterial Hypertension 2003 0.0
Japanese Society of Hypertension Guidelines for the Management of Hypertension (JSH 2004) 0.0
2016 – European Society of Hypertension Guidelines for the management of high blood pressure in children and adolescents 0.1
2015 Oman Heart Association Guidelines for the Management of Hypertension 0.0
2013 ESH/ESC GUIDELINES FOR THE MANAGEMENT OF ARTERIAL HYPERTENSION 0.0


In [31]:
# Configure pandas display settings
import pandas as pd
pd.set_option('display.max_colwidth', None)  # Show full text in columns
pd.set_option('display.max_rows', None)      # Show all rows
pd.set_option('display.width', None)         # Auto-adjust display width
pd.set_option('display.max_columns', None)   # Show all columns

def compare_search_methods(query, top_k=10, year_range={"start": 2015, "end": 2025}):
    """
    Compare search results between FAISS and API approaches
    """
    # 1. Get FAISS results
    faiss_results, _ = search_engine.search(query, top_k=top_k)
    faiss_ranked = FAISSSearch.rank_results(faiss_results, p=0.9)
    df_faiss = pd.DataFrame(faiss_ranked)[['title', 'year', 'Score', 'recency_score', 'final_score']]
    
    # 2. Get API results
    api_payload = {
        "query": query,
        "limit": top_k,
        "filters": {
            "yearRange": year_range,
            "language": "en"
        }
    }
    
    response = requests.post(API_URL, json=api_payload, headers={"Content-Type": "application/json"})
    if response.status_code == 200:
        api_results = response.json()['eval_results']
        df_api = pd.DataFrame(api_results)[['title', 'score']]
        df_api = df_api.rename(columns={'score': 'api_score'})
    else:
        print(f"API Error: {response.status_code}")
        return None
    
    # 3. Compare results
    print(f"Query: '{query}'\n")
    print("=== FAISS Top Results ===")
    display(df_faiss.head())
    print("\n=== API Top Results ===")
    display(df_api.head())
    
    # 4. Analyze overlap
    faiss_titles = set(df_faiss['title'].str.lower())
    api_titles = set(df_api['title'].str.lower())
    overlap = faiss_titles.intersection(api_titles)
    
    print(f"\nResults Analysis:")
    print(f"Total overlapping results: {len(overlap)}")
    print(f"Results unique to FAISS: {len(faiss_titles - api_titles)}")
    print(f"Results unique to API: {len(api_titles - faiss_titles)}")
    
    return df_faiss, df_api

# Example usage
query = "The treatment of sarcoidosis: translating the European respiratory guidelines into clinical practice "
faiss_results, api_results = compare_search_methods(query, top_k=10)

Query: 'The treatment of sarcoidosis: translating the European respiratory guidelines into clinical practice '

=== FAISS Top Results ===


,title,year,Score,recency_score,final_score
0,Do we practice what we preach? Implementation of cardiovascular prevention strategies in 13 European countries between 2011 and 2021: a statement of the European Association of Preventive Cardiology of the ESC,2023,0.073,0.8,0.1457
1,Management of Psoriasis With Topicals: Applying the 2020 AAD-NPF Guidelines of Care to Clinical Practice,2022,0.078,0.7,0.1402
2,European Respiratory Society Guidelines for the Management of Bronchiectasis in Children and Adolescents,2021,0.075,0.6,0.1275
3,Comment on “Diagnosis and treatment of basal cell carcinoma: European consensus–based interdisciplinary guidelines”,2020,0.080,0.5,0.1220
4,Monitoring of inflammatory bowel disease in 2019: A French consensus for clinical practice,2020,0.077,0.5,0.1193



=== API Top Results ===


,title,api_score
0,The treatment of sarcoidosis: translating the European respiratory guidelines into clinical practice,0.572
1,Comment on the European Respiratory Society pulmonary alveolar proteinosis guidelines,0.520
2,European Respiratory Society Guidelines for the Management of Bronchiectasis in Children and Adolescents,0.463
3,European Respiratory Society clinical practice guideline on symptom management for adults with serious respiratory illness,0.389
4,Clinical Practice Guideline: Nosebleed (Epistaxis),0.327



Results Analysis:
Total overlapping results: 1
Results unique to FAISS: 9
Results unique to API: 11
